In [1]:
path = '/notebook/kaggle/'
PROJ_HOME_DIR = path + 'data/ultrasound/'
DATA_HOME_DIR = PROJ_HOME_DIR # + 'sample/'

results_path = DATA_HOME_DIR + 'results/'
test_path = DATA_HOME_DIR + 'test/'
valid_path = DATA_HOME_DIR + 'valid/'
train_path = DATA_HOME_DIR + 'train/'

In [5]:
%run -i $PROJ_HOME_DIR'data.py'

------------------------------
Creating training images...
------------------------------
Done: 0/5635 images


/usr/local/lib/python2.7/dist-packages/skimage/external/tifffile/tifffile.py:2611: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)


Done: 1000/5635 images
Done: 2000/5635 images
Done: 3000/5635 images
Done: 4000/5635 images
Done: 5000/5635 images
Loading done.
Saving to .npy files done.
------------------------------
Creating test images...
------------------------------
Done: 0/5508 images
Done: 1000/5508 images
Done: 2000/5508 images
Done: 3000/5508 images
Done: 4000/5508 images
Done: 5000/5508 images
Loading done.
Saving to .npy files done.


In [ ]:
!pip install scikit-image
!pip install kaggle-cli # Kaggle command-line interface

In [2]:
import json

with open(path+'kaggle.txt') as json_file:  
    kg_data = json.load(json_file)
    
username = kg_data['username']
password = kg_data['password']

In [3]:
!kg config -g -u $username -p $password -c 'ultrasound-nerve-segmentation'

'''
%cd $DATA_HOME_DIR
!kg download
'''

'\n%cd $DATA_HOME_DIR\n!kg download\n'

In [6]:
!echo $CUDA_VISIBLE_DEVICES

0


In [7]:
from __future__ import print_function

import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K

#import sys
#sys.path.append('/notebook/kaggle/data/ultrasound')
#from data import load_train_data, load_test_data

K.set_image_data_format('channels_last')  # TF dimension ordering in this code

img_rows = 96
img_cols = 96

Using TensorFlow backend.


In [8]:
def preprocess(imgs):
    ''' Convert images to np arrays of 8-bit unsigned integers and standardize shape '''
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

This competition is evaluated on the mean [Dice coefficient](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient). The Dice coefficient can be used to compare the pixel-wise agreement between a predicted segmentation and its corresponding ground truth.

In [9]:
# Dice coefficient between actual and predicted pixels
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))

# Loss of Dice coefficient between actual and predicted pixels
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

Create and compile [**u-net model**](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/): a network that relies on the strong use of data augmentation to use the available annotated samples more efficiently. The architecture consists of a contracting path to capture context and a symmetric expanding path that enables precise localization.
[More info...](https://arxiv.org/abs/1505.04597)

In [10]:
inputs = Input((img_rows, img_cols, 1))
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

model = Model(inputs=[inputs], outputs=[conv10])

model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
model_checkpoint = ModelCheckpoint(results_path+'weights.h5', monitor='val_loss', save_best_only=True)

In [11]:
# Load and preprocess train data
imgs_train = preprocess(np.load(DATA_HOME_DIR+'imgs_train.npy'))
imgs_mask_train = preprocess(np.load(DATA_HOME_DIR+'imgs_mask_train.npy'))

# Normalize data
imgs_train = imgs_train.astype('float32')
mean = np.mean(imgs_train)  # Mean for data centering
std = np.std(imgs_train)  # std for data normalization

imgs_train -= mean
imgs_train /= std

imgs_mask_train = imgs_mask_train.astype('float32')
imgs_mask_train /= 255.  # scale masks to [0, 1]

/usr/local/lib/python2.7/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Train model

In [13]:
model.fit(imgs_train, imgs_mask_train, batch_size=32, epochs=20, verbose=1, shuffle=True,
          validation_split=0.2,
          callbacks=[model_checkpoint])

Train on 4508 samples, validate on 1127 samples
Epoch 1/20
4508/4508 [==============================] - 18s - loss: -0.0842 - dice_coef: 0.0842 - val_loss: -0.2030 - val_dice_coef: 0.2030
Epoch 2/20
4508/4508 [==============================] - 17s - loss: -0.2501 - dice_coef: 0.2501 - val_loss: -0.2486 - val_dice_coef: 0.2486
Epoch 3/20
4508/4508 [==============================] - 18s - loss: -0.2642 - dice_coef: 0.2642 - val_loss: -0.2462 - val_dice_coef: 0.2462
Epoch 4/20
4508/4508 [==============================] - 18s - loss: -0.2743 - dice_coef: 0.2743 - val_loss: -0.2655 - val_dice_coef: 0.2655
Epoch 5/20
4508/4508 [==============================] - 18s - loss: -0.2802 - dice_coef: 0.2802 - val_loss: -0.2688 - val_dice_coef: 0.2688
Epoch 6/20
4508/4508 [==============================] - 17s - loss: -0.2880 - dice_coef: 0.2880 - val_loss: -0.2526 - val_dice_coef: 0.2526
Epoch 7/20
4508/4508 [==============================] - 18s - loss: -0.2998 - dice_coef: 0.2998 - val_loss: -0.2

Make predictions on test data

In [47]:
# Load and preprocess test data
imgs_test = preprocess(np.load(DATA_HOME_DIR+'imgs_test.npy'))
imgs_id_test = np.load(DATA_HOME_DIR+'imgs_id_test.npy')

imgs_test = imgs_test.astype('float32')
imgs_test -= mean
imgs_test /= std

In [48]:
imgs_test[1,:,:,0]

array([[ 1.11051011,  1.1282295 ,  1.05735195, ...,  2.54577971,
         2.74069285,  2.74069285],
       [ 1.50033641,  1.64209139,  1.41173947, ...,  1.64209139,
         1.21682632,  0.95103574],
       [ 1.37630081,  1.41173947,  1.2522651 , ..., -1.19300866,
        -1.12213111, -1.12213111],
       ..., 
       [-0.34247866, -0.25388178, -0.28932053, ..., -1.60055435,
        -1.60055435, -1.56511557],
       [ 0.11822511,  0.04734761, -0.05896864, ..., -1.58283496,
        -1.58283496, -1.58283496],
       [ 0.11822511, -0.00581052, -0.18300427, ..., -1.60055435,
        -1.60055435, -1.61827362]], dtype=float32)

In [49]:
# Load saved weights
model.load_weights(results_path+'weights.h5')

# Predict masks using test data
imgs_mask_test = model.predict(imgs_test, verbose=1)
np.save(DATA_HOME_DIR+'imgs_mask_test.npy', imgs_mask_test)

5472/5508 [============================>.] - ETA: 0s

In [11]:
# Save predicted masks to file
pred_dir = results_path+'preds'
if not os.path.exists(pred_dir):
    os.mkdir(pred_dir)
for image, image_id in zip(imgs_mask_test, imgs_id_test):
    image = (image[:, :, 0] * 255.).astype(np.uint8)
    imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/164_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1953_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/106_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3774_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2394_pred.png is a low contrast image
  warn('%s is a low co

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2218_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1111_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/5478_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/914_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4460_pred.png is a low contrast image
  warn('%s is a low c

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/328_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2792_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2338_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1951_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2513_pred.png is a low contrast image
  warn('%s is a low c

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/96_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1444_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/509_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3782_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3756_pred.png is a low contrast image
  warn('%s is a low con

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3629_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4234_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1086_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2889_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1836_pred.png is a low contrast image
  warn('%s is a low 

...

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3128_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1897_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3355_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/231_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2883_pred.png is a low contrast image
  warn('%s is a low c

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/5354_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/317_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4017_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3810_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4540_pred.png is a low contrast image
  warn('%s is a low c

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3440_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3002_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/560_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4604_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/723_pred.png is a low contrast image
  warn('%s is a low co

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/5364_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/878_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4533_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1235_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1105_pred.png is a low contrast image
  warn('%s is a low c

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/1303_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/3148_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2180_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/4576_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /notebook/kaggle/data/ultrasound/results/preds/2529_pred.png is a low contrast image
  warn('%s is a low 

In [50]:
imgs_mask_test[0]

array([[[  1.52358245e-02],
        [  3.30209895e-03],
        [  7.09659886e-04],
        ..., 
        [  9.52795672e-04],
        [  6.22808002e-03],
        [  1.89848244e-02]],

       [[  1.36398512e-03],
        [  9.80422628e-05],
        [  2.91977667e-05],
        ..., 
        [  9.31367231e-06],
        [  3.89871944e-04],
        [  2.47654016e-03]],

       [[  3.39793653e-04],
        [  8.02815430e-06],
        [  8.19788184e-08],
        ..., 
        [  5.59502439e-07],
        [  1.81317719e-05],
        [  1.42618106e-03]],

       ..., 
       [[  3.97248333e-03],
        [  3.30129435e-04],
        [  2.84489743e-05],
        ..., 
        [  1.73483295e-05],
        [  1.60163363e-05],
        [  6.29183603e-04]],

       [[  1.79719571e-02],
        [  3.06666759e-03],
        [  3.94697912e-04],
        ..., 
        [  3.41146369e-04],
        [  1.59852195e-03],
        [  6.75357273e-03]],

       [[  4.26582731e-02],
        [  2.17901897e-02],
        [  

In [51]:
#
imgs_test = np.load(DATA_HOME_DIR+'imgs_mask_test.npy')
imgs_test[0]

array([[[  1.52358245e-02],
        [  3.30209895e-03],
        [  7.09659886e-04],
        ..., 
        [  9.52795672e-04],
        [  6.22808002e-03],
        [  1.89848244e-02]],

       [[  1.36398512e-03],
        [  9.80422628e-05],
        [  2.91977667e-05],
        ..., 
        [  9.31367231e-06],
        [  3.89871944e-04],
        [  2.47654016e-03]],

       [[  3.39793653e-04],
        [  8.02815430e-06],
        [  8.19788184e-08],
        ..., 
        [  5.59502439e-07],
        [  1.81317719e-05],
        [  1.42618106e-03]],

       ..., 
       [[  3.97248333e-03],
        [  3.30129435e-04],
        [  2.84489743e-05],
        ..., 
        [  1.73483295e-05],
        [  1.60163363e-05],
        [  6.29183603e-04]],

       [[  1.79719571e-02],
        [  3.06666759e-03],
        [  3.94697912e-04],
        ..., 
        [  3.41146369e-04],
        [  1.59852195e-03],
        [  6.75357273e-03]],

       [[  4.26582731e-02],
        [  2.17901897e-02],
        [  

Generate submission file

In [52]:
from skimage.transform import resize

image_rows = 420
image_cols = 580

def prep(img):
    img = img.astype('float32')
    img = (img > 0.5).astype(np.uint8)  # threshold
    img = resize(img, (image_cols, image_rows), preserve_range=True)
    return img

def run_length_enc(label):
    from itertools import chain
    x = label.transpose().flatten()
    y = np.where(x > 0)[0]
    if len(y) < 10:  # consider as empty
        return ''
    z = np.where(np.diff(y) > 1)[0]
    start = np.insert(y[z+1], 0, y[0])
    end = np.append(y[z], y[-1])
    length = end - start
    res = [[s+1, l+1] for s, l in zip(list(start), list(length))]
    res = list(chain.from_iterable(res))
    return ' '.join([str(r) for r in res])

# Create .csv for submission
#_, imgs_id_test = load_test_data()
imgs_test = np.load(DATA_HOME_DIR+'imgs_mask_test.npy')

argsort = np.argsort(imgs_id_test)
imgs_id_test = imgs_id_test[argsort]
imgs_test = imgs_test[argsort]

total = imgs_test.shape[0]
ids = []
rles = []
for i in range(10): # 'total'
    img = imgs_test[i, 0]
    img = prep(img)
    rle = run_length_enc(img)

    rles.append(rle)
    ids.append(imgs_id_test[i])

    if i % 1000 == 0:
        print('{}/{}'.format(i, total))

print(rles)

0/5508
['', '', '', '', '', '', '', '', '', '']


In [53]:
imgs_test[0]

array([[[  1.59769244e-02],
        [  3.57137946e-03],
        [  7.88879406e-04],
        ..., 
        [  1.00855296e-02],
        [  2.72668395e-02],
        [  6.10860549e-02]],

       [[  1.62224716e-03],
        [  1.43088953e-04],
        [  3.84435625e-05],
        ..., 
        [  3.50307819e-04],
        [  4.56963759e-03],
        [  1.69295408e-02]],

       [[  4.43083700e-04],
        [  8.46165403e-06],
        [  8.85961171e-08],
        ..., 
        [  4.98309200e-05],
        [  5.56467741e-04],
        [  9.28779412e-03]],

       ..., 
       [[  8.14990417e-05],
        [  1.17069783e-06],
        [  1.85878370e-08],
        ..., 
        [  3.65768110e-11],
        [  4.09454901e-11],
        [  8.78741346e-08]],

       [[  1.10848248e-03],
        [  5.07821132e-05],
        [  1.56613748e-06],
        ..., 
        [  2.41903635e-08],
        [  8.42114900e-07],
        [  1.74839734e-05]],

       [[  5.00567723e-03],
        [  1.59097824e-03],
        [  

In [43]:
prep(imgs_test[5])

array([[[ 0.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 0.],
        [ 0.]],

       [[ 0.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 0.],
        [ 0.]],

       [[ 0.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 0.],
        [ 0.]],

       ..., 
       [[ 0.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 0.],
        [ 0.]],

       [[ 0.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 0.],
        [ 0.]],

       [[ 0.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 0.],
        [ 0.]]])

In [13]:
first_row = 'img,pixels'
file_name = results_path+'submission.csv'

with open(file_name, 'w+') as f:
    f.write(first_row + '\n')
    for i in range(total):
        s = str(ids[i]) + ',' + rles[i]
        f.write(s + '\n')

In [16]:
# ! kg submit $file_name # Un-comment to submit

list index out of range
